In [1]:
import numpy as np
import math
from scipy.optimize import linear_sum_assignment


def euclidianDistance(p1,p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def polar_to_cartesian(magnitude, angle_rad):
    x = magnitude * np.cos(angle_rad)
    y = magnitude * np.sin(angle_rad)
    return np.array([x, y])

def cosine_similarity(v1Polar, v2Polar):
    v1Cartesian = polar_to_cartesian(v1Polar[0], v1Polar[1])
    v2Cartesian = polar_to_cartesian(v2Polar[0], v2Polar[1])
    return np.dot(v1Cartesian, v2Cartesian) / (np.linalg.norm(v1Cartesian) * np.linalg.norm(v2Cartesian))

def Vector(fromCoord, toCoord, latency):
        if latency < 1e-5:
            return 0, 0
        else:
            magnitude = euclidianDistance(fromCoord, toCoord) / latency
            radians = math.atan2(toCoord[1] - fromCoord[1], toCoord[0] - fromCoord[0])
            if radians < 0: #to get positive radian values (to get true fraction)
                radians = (2*math.pi)+radians
            return magnitude, radians


'''
vid upptäckt markera med röd ring
om fler än 10 koordinater registreras på samma id, kör på som vanligt
'''

class OBJECT():
    def __init__(self, id, initialCoord, time):
        self.id = id
        self.trajectory = []
        self.vectors = []
        self.avg_vector_short = None
        self.avg_vector_long = None
        self.diff_avg_magnitude = 0
        self.clock = time
        self.latency = 0
        self.fresh = False

        self.preliminary_trajectory = [(initialCoord, time)]
        self.nmbr_of_coord_id = 0
        '''
        to track ALL data (CSV)
        '''
        self.data = []

    def addCoordinate(self, coordinate, time):
        if self.nmbr_of_coord_id < 10:
            self.preliminary_trajectory.append((coordinate, time))
        else:
            if self.nmbr_of_coord_id == 10:
                for preliminary_coord in self.preliminary_trajectory:
                    self.trajectory.append(preliminary_coord)
                    self.setVector(preliminary_coord[0])
                self.preliminary_trajectory = []
            self.trajectory.append((coordinate, time))
        self.nmbr_of_coord_id += 1
        self.latency = time - self.clock
        self.clock = time
        if len(self.trajectory) > 1:
            self.setVector(coordinate)
    def setFresh(self, bool):
        self.fresh = bool
    def getData(self):
        return self.data
    def getLatestCoord(self):
        if len(self.preliminary_trajectory) == 0:
            return self.trajectory[len(self.trajectory)-1]
        else:
            return self.preliminary_trajectory[len(self.preliminary_trajectory)-1]
    def getEarliestCoord(self):
        if len(self.preliminary_trajectory) == 0:
            return self.trajectory[0], 'track'
        else:
            return self.preliminary_trajectory[0], 'pre'
    def getLatestAvgVectorShort(self):
        return self.avg_vector_short
    def getLatestAvgVectorLong(self):
        return self.avg_vector_long
    def setVector(self, coord):
        fromCoord = self.trajectory[len(self.trajectory)-2][0]
        toCoord = self.getLatestCoord()[0]
        magnitude, radians = Vector(fromCoord, toCoord, self.latency)
        self.vectors.append((magnitude, radians))
        
        consider_nmbr_of_vectors_short = 10
        consider_nmbr_of_vectors_long = 50
        actualVectorsForAvgShort, actualVectorsForAvgLong = 0, 0
        x_short, y_short = 0, 0
        x_long, y_long = 0, 0
        for index,_ in enumerate(reversed(self.vectors)):
            current_vector = self.vectors[len(self.vectors)-(1+index)] 
            magnitude, angle = current_vector[0], current_vector[1]  
            if index < consider_nmbr_of_vectors_short: 
                x_short += magnitude * math.cos(angle)
                y_short += magnitude * math.sin(angle)
                actualVectorsForAvgShort = index + 1
            else:
                pass
            if index < consider_nmbr_of_vectors_long:
                x_long += magnitude * math.cos(angle)
                y_long += magnitude * math.sin(angle)
                actualVectorsForAvgLong = index + 1
            else: 
                break

        x_avg_short = x_short / (actualVectorsForAvgShort)
        y_avg_short = y_short / (actualVectorsForAvgShort)
        avg_magnitude_short = math.hypot(x_avg_short, y_avg_short)
        avg_radians_short = math.atan2(y_avg_short, x_avg_short)
        self.avg_vector_short = (avg_magnitude_short,avg_radians_short)


        if actualVectorsForAvgLong > 40 and actualVectorsForAvgLong <= consider_nmbr_of_vectors_long:
            x_avg_long = x_long / (actualVectorsForAvgLong)
            y_avg_long = y_long / (actualVectorsForAvgLong)
            avg_magnitude_long = math.hypot(x_avg_long, y_avg_long)
            avg_radians_long = math.atan2(y_avg_long, x_avg_long)
            self.avg_vector_long = (avg_magnitude_long, avg_radians_long)
            self.diff_avg_magnitude = math.sqrt((avg_magnitude_long - avg_magnitude_short)**2)
            self.data.append([self.id, self.diff_avg_magnitude, self.latency, self.avg_vector_short[0], self.avg_vector_short[1],
                                self.avg_vector_long[0], self.avg_vector_long[1], coord[0], coord[1], self.clock])
        self.setFresh(True)
'''
förbättringar:
                    - max ett objekt per closestID per frame (undvika att en blir en annan)
                    - endast några index bakåt (för att objekt som korsar gammal färdväg ej ska byta)

                    vektor - expected next position? drönare lär vara enklare att förutspå än fåglar (värt att klassa på?)
                    plotta vektor i raw : 
                    *** x1 = x0 + magnitude * math.cos(angle_rad) ***
                    *** y1 = y0 + magnitude * math.sin(angle_rad) ***
'''

class TRACKER():
    def __init__(self, maxDistance):
        self.maxDistance = maxDistance
        self.trackedObjects = {}
        self.detectedObjects = {}
        self.assignID = 1
        self.patience = 5 #s

        self.objectsToPop = []
        self.newIteration = True

        self.coordID = 0
        self.contestorNewCoords = []
        self.contestorNewIDs = []
        self.contestorObjectsCoords = []
        self.contestorObjectsIDs = []

    def reset(self):
        self.contestorNewCoords = []
        self.contestorNewIDs = []
        self.contestorObjectsCoords = []
        self.contestorObjectsIDs = []
        self.coordID = 0
        self.objectsToPop = []
        self.newIteration = True

    def track(self, coord, time):
        ''' 
        coord : ((x, y), time) 
        '''
        minDistance = 1e9
        vectorCosineSimularity = 1e9
        closestID = None
        mostSimilarID = None
        assignID = None
        assignScore = 0
        contestor = False
        '''
        mindiff lista som sorterar tillräckligt nära objects diffar 
        prob distr på mag, rad och dist
        '''
        for trackedObject in self.trackedObjects:
            #trackedObject = self.trackedObjects[trackedObject].id
            latency = time - self.trackedObjects[trackedObject].clock
            if latency > self.patience and len(self.trackedObjects[trackedObject].trajectory) < 90:
                if trackedObject not in self.objectsToPop:
                    self.objectsToPop.append(trackedObject)
            else:
                latest = self.trackedObjects[trackedObject].getLatestCoord()
                latestCoord, latestCoordAtTime = latest[0], latest[1]
                dist = euclidianDistance(coord, latestCoord)
                if dist < self.maxDistance: 
                    if trackedObject not in self.contestorObjectsIDs:
                        self.contestorObjectsCoords.append([latestCoord[0], latestCoord[1]])
                        self.contestorObjectsIDs.append(trackedObject)
                    contestor = True
        if not contestor:
            self.trackedObjects[self.assignID] = OBJECT(self.assignID, coord, time)
            self.detectedObjects[self.assignID] = self.trackedObjects[self.assignID]
            self.assignID += 1
            pass
        else:
            if self.coordID not in self.contestorNewIDs:
                self.contestorNewCoords.append([coord[0], coord[1]])
                self.contestorNewIDs.append(self.coordID)
                self.coordID += 1
        '''
        fixa så avstånd viktigare än vector. men inom ett visst threshhold ska vectorliknelse ha betydelse
        '''

    def assign(self, time):
        if not self.contestorObjectsCoords or not self.contestorNewCoords:
            for coord in self.contestorNewCoords:
                self.trackedObjects[self.assignID] = OBJECT(self.assignID, coord, time)
                self.detectedObjects[self.assignID] = self.trackedObjects[self.assignID]
                self.assignID += 1
            self.reset()
            return

        object_id_map = self.contestorObjectsIDs
        coord_list = self.contestorNewCoords

        num_objects = len(object_id_map)
        num_coords = len(coord_list)
        max_dim = max(num_objects, num_coords)

        cost_matrix = np.full((max_dim, max_dim), fill_value=1e6)  # pad if nec

        for i, last_coord in enumerate(self.contestorObjectsCoords):
            object_id = object_id_map[i]
            obj = self.trackedObjects[object_id]

            latency = obj.latency if obj.latency > 1e-5 else 1e-5
            long_vector = obj.getLatestAvgVectorLong()
            short_vector = obj.getLatestAvgVectorShort()

            if long_vector is not None and long_vector[0] > 0:
                predicted_offset = polar_to_cartesian(long_vector[0]*latency, long_vector[1]*latency)
                predicted_vector = long_vector
            #elif short_vector is not None and short_vector[0] > 0:
             #   predicted_offset = polar_to_cartesian(short_vector[0]*latency, short_vector[1]*latency)
              #  predicted_vector = short_vector
            else:
                predicted_offset = np.array([0, 0])
                predicted_vector = None

            predicted_position = np.array(last_coord) + predicted_offset

            for j, coord in enumerate(coord_list):
                coord = np.array(coord)
                distance_to_predicted = np.linalg.norm(predicted_position - coord)

                if distance_to_predicted > self.maxDistance:
                    cost_matrix[i, j] = 1e6
                    continue

                observed_vector = Vector(last_coord, coord, latency)

                if predicted_vector is None or observed_vector[0] <= 0:
                    angle_penalty = 1.0
                else:
                    angle_penalty = 1 - cosine_similarity(predicted_vector, observed_vector)

                alpha = 3.0  # distance
                beta = 7.0  # radians
                vector_confidence = min(len(obj.vectors) / 10, 1.0)
                total_cost = (1 - vector_confidence) * distance_to_predicted + vector_confidence * (alpha * distance_to_predicted + beta * angle_penalty)
                cost_matrix[i, j] = total_cost

        row_ind, col_ind = linear_sum_assignment(cost_matrix)

        assigned_coords = set()
        assigned_objects = set()

        for r, c in zip(row_ind, col_ind):
            if r >= num_objects or c >= num_coords:
                continue
            if cost_matrix[r, c] >= 1e6:
                continue  

            object_id = object_id_map[r]
            coord = coord_list[c]
            if object_id in self.trackedObjects:
                self.trackedObjects[object_id].addCoordinate(coord, time)
                assigned_coords.add(c)
                assigned_objects.add(object_id)

        for i, coord in enumerate(coord_list):
            if i not in assigned_coords:
                self.trackedObjects[self.assignID] = OBJECT(self.assignID, coord, time)
                self.detectedObjects[self.assignID] = self.trackedObjects[self.assignID]
                self.assignID += 1

    def popOldTracks(self):
        for objectToPop in self.objectsToPop:
            self.trackedObjects.pop(objectToPop)





In [2]:
import numpy as np

class RNN():
    def __init__(self):
        self.LSTM = None
        self.prev = [None, None]

        self.magnitude = None
        self.unitVector = None
        pass
    def forward(self):
        pass
    def backwards(self):
        pass
    def update(self, position):
        position = np.array([position[0], position[1]], dtype=np.float32)
        if self.prev[0] != None:
            vector = position - self.prev                      # vectorize
            # SPEED
            self.magnitude = np.sqrt(np.dot(vector, vector))   # dot and sqrt for magnitude   
            # DIRECTION 
            self.unitVector = vector / self.magnitude if self.magnitude != 0 else np.zeros(2, dtype=np.float32)
            pass

        self.prev = np.array([position[0], position[1]], dtype=np.float32)
        return (self.magnitude, self.unitVector)

In [3]:
import cv2
import math
import numpy as np

def directional_cone(frame, x0, y0, x1, y1, spread_degrees=45, color=(0, 0, 255), alpha=0.3):
    dx = x1 - x0
    dy = y1 - y0
    direction = math.atan2(dy, dx)
    length = math.hypot(dx, dy)

    # Convert spread angle to radians
    spread = math.radians(spread_degrees)

    # Compute left and right points of the cone base
    left = (
        int(x0 + length * math.cos(direction - spread / 2)),
        int(y0 + length * math.sin(direction - spread / 2))
    )
    right = (
        int(x0 + length * math.cos(direction + spread / 2)),
        int(y0 + length * math.sin(direction + spread / 2))
    )

    # Define triangle points: tip, left, right
    pts = np.array([[x0, y0], left, right], np.int32)
    pts = pts.reshape((-1, 1, 2))

    # Draw transparent triangle
    overlay = frame.copy()
    cv2.fillPoly(overlay, [pts], color)

    # Blend overlay with original frame
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)


In [4]:
import os
import cv2
import time
import math
import csv
from ultralytics import YOLO
from queue import Queue

# === Constants ===
SEQUENCE_LENGTH = 90
MODEL_PATH = r"C:\Users\joelw\anaconda3\dronebird\best.pt"
CONFIDENCE_THRESHOLD = 0.2
SCALE_PERCENT = 30
SHOW_FPS = True
SHOW_DISPLAY = True
SHOW_VECTOR_SHORT = True
SHOW_VECTOR_LONG = True
SHOW_DIRECTIONAL_CONES = True
SHOW_TRAJECTORY = True
SHOW_ID_TRACKED = True
SHOW_ID_DETECTED = False
SHOW_YOLO_BOUNDINGBOXES = False
SAVE_OUTPUT = False

# === Colors ===
ORANGE = (0,150,220)
BLUE = (175, 0, 0)
GREEN = (0, 150, 0)
RED = (0, 0, 175)
COLORS = [GREEN, RED, BLUE, ORANGE]

# === Export function ===
def export_sequences(tracked_objects, output_path="all_sequences.csv", label=0):
    rows = []
    sample_entry = None
    for obj in tracked_objects.values():
        data = obj.getData()
        if len(data) < SEQUENCE_LENGTH:
            continue
        if sample_entry is None and data:
            sample_entry = data[0]
        for i in range(0, len(data) - SEQUENCE_LENGTH + 1, SEQUENCE_LENGTH):
            chunk = data[i:i + SEQUENCE_LENGTH]
            flattened = []
            for entry in chunk:
                flattened.extend(entry)
            flattened.append(label)
            rows.append(flattened)
    if not rows:
        print("No valid sequences found.")
        return
    if sample_entry is None:
        print("No sample data to generate headers.")
        return
    feature_names = ["ID", "Diff_Avg_Magnitude", "Latency",
                     "Avg_Vector_Short_Magnitude", "Avg_Vector_Short_Angle",
                     "Avg_Vector_Long_Magnitude", "Avg_Vector_Long_Angle",
                     "X", "Y", "Timestamp"]
    headers = []
    for i in range(SEQUENCE_LENGTH):
        headers.extend([f"{name}_{i}" for name in feature_names])
    headers.append("Label")
    write_header = not os.path.exists(output_path) or os.path.getsize(output_path) == 0
    with open(output_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        if write_header:
            writer.writerow(headers)
        writer.writerows(rows)
    print(f"Saved {len(rows)} sequence(s) to {output_path}")

# === Main processing function ===
def process_video(video_path, label):
    print(f"\n=== Processing: {video_path} ===")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return

    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to read first frame.")
        cap.release()
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(frame.shape[1] * SCALE_PERCENT / 100)
    height = int(frame.shape[0] * SCALE_PERCENT / 100)

    # Load model and tracker
    model = YOLO(MODEL_PATH)
    model.fuse()
    if model.device.type == 'cuda':
        model.half()

    rnn = RNN()
    tracker = TRACKER(40)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None
    if SAVE_OUTPUT:
        output_path = f"output_videos/{os.path.splitext(os.path.basename(video_path))[0]}_out.mp4"
        out = cv2.VideoWriter(output_path, fourcc, original_fps, (width, height))

    fps_counter = []
    shutdown = False

    while True:
        ret, full_frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(full_frame, (width, height))
        start_time = time.time()

        current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

        results = model(frame, conf=CONFIDENCE_THRESHOLD, iou=0.3, verbose = False)
        annotated = results[0].plot() if SHOW_YOLO_BOUNDINGBOXES else frame

        for result in results:
            boxes = result.boxes
            if boxes is not None:
                centers = boxes.xywh[:, :2]
                for center in centers:
                    coord = tuple(map(int, center))
                    tracker.track(coord, current_time)
                tracker.assign(current_time)
                tracker.popOldTracks()
                tracker.reset()

        for id in tracker.trackedObjects:
            obj = tracker.trackedObjects[id]
            for coord in obj.trajectory:
                if SHOW_TRAJECTORY:
                    cv2.circle(annotated, coord[0], 1, COLORS[id % 4])
            if obj.avg_vector_long and obj.fresh:
                mag, angle = obj.avg_vector_long
                x0, y0 = obj.getLatestCoord()[0]
                x1 = int(x0 + mag * math.cos(angle))
                y1 = int(y0 + mag * math.sin(angle))
                if SHOW_VECTOR_LONG:
                    cv2.line(annotated, (x0, y0), (x1, y1), (0, 0, 255), 1)
                if SHOW_DIRECTIONAL_CONES:
                    directional_cone(annotated, x0, y0, x1, y1)
            if obj.avg_vector_short and SHOW_VECTOR_SHORT and obj.fresh:
                mag, angle = obj.avg_vector_short
                x0, y0 = obj.getLatestCoord()[0]
                x1 = int(x0 + mag * math.cos(angle))
                y1 = int(y0 + mag * math.sin(angle))
                cv2.arrowedLine(annotated, (x0, y0), (x1, y1), (255, 0, 0), 1)
            coord, origin = obj.getEarliestCoord()
            if (SHOW_ID_TRACKED and origin == 'track') or SHOW_ID_DETECTED:
                x, y = coord[0]  # Correct: extract from ((x, y), timestamp)
                cv2.putText(annotated, str(id), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, COLORS[id % 4], 1)
            obj.setFresh(False)

        if SHOW_FPS:
            fps = 1.0 / (time.time() - start_time)
            fps_counter.append(fps)
            cv2.putText(annotated, f"{fps:.1f} FPS", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 200), 2)

        if SHOW_DISPLAY:
            cv2.imshow("YOLOv8 Detection", annotated)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown = True
                break

        if SAVE_OUTPUT and out:
            out.write(annotated)

    cap.release()
    if out:
        out.release()
    if SHOW_DISPLAY:
        cv2.destroyAllWindows()

    if SHOW_FPS and fps_counter:
        print(f"Average FPS: {sum(fps_counter)/len(fps_counter):.2f}")

    export_sequences(tracker.trackedObjects, label=label)


WARNING ⚠️ torchvision==0.20 is incompatible with torch==2.4.
Run 'pip install torchvision==0.19' to fix torchvision or 'pip install -U torch torchvision' to update both.
For a full compatibility table see https://github.com/pytorch/vision#installation


In [6]:
import os
print(os.path.exists("anaconda3/dronebird/raw/Birds"))

True


In [7]:
import os
import re

VIDEO_DIR = r"C:\Users\joelw\anaconda3\dronebird\raw\Birds"  # FIXED: only relative from current folder
LABEL = 0  # 1 = Drone, 0 = Bird

def natural_key(filename):
    return [int(s) if s.isdigit() else s.lower() for s in re.split(r'(\d+)', filename)]

try:
    print("Current working directory:", os.getcwd())
    print("Looking in directory:", VIDEO_DIR)
    files = os.listdir(VIDEO_DIR)
    print("Files found:", files)

    video_files = sorted([
        os.path.join(VIDEO_DIR, f)
        for f in files
        if f.lower().endswith(".mov")  # Case-insensitive
    ], key=lambda x: natural_key(os.path.basename(x)))

    if not video_files:
        print("No video files found!")

    for path in video_files:
        print("Calling process_video...")
        print(f"\n=== Running on: {path} ===")
        process_video(path, LABEL)
        print("Finished calling process_video.")

except FileNotFoundError as e:
    print(f"Error: {e}")


Current working directory: C:\Users\joelw
Looking in directory: C:\Users\joelw\anaconda3\dronebird\raw\Birds
Files found: ['B_1.MOV', 'B_10.MOV', 'B_100.MOV', 'B_101.MOV', 'B_102.MOV', 'B_103.MOV', 'B_104.MOV', 'B_105.MOV', 'B_106.MOV', 'B_107.MOV', 'B_108.MOV', 'B_109.MOV', 'B_11.MOV', 'B_12.MOV', 'B_13.MOV', 'B_14.MOV', 'B_15.MOV', 'B_16.MOV', 'B_17.MOV', 'B_18.MOV', 'B_19.MOV', 'B_2.MOV', 'B_20.MOV', 'B_21.MOV', 'B_22.MOV', 'B_23.MOV', 'B_24.MOV', 'B_25.MOV', 'B_26.MOV', 'B_27.MOV', 'B_28.MOV', 'B_29.MOV', 'B_3.MOV', 'B_30.MOV', 'B_31.MOV', 'B_32.MOV', 'B_33.MOV', 'B_34.MOV', 'B_35.MOV', 'B_36.MOV', 'B_37.MOV', 'B_38.MOV', 'B_39.MOV', 'B_4.MOV', 'B_40.MOV', 'B_41.MOV', 'B_42.MOV', 'B_43.MOV', 'B_44.MOV', 'B_45.MOV', 'B_46.MOV', 'B_47.MOV', 'B_48.MOV', 'B_49.MOV', 'B_5.MOV', 'B_50.MOV', 'B_51.MOV', 'B_52.MOV', 'B_53.MOV', 'B_54.MOV', 'B_55.MOV', 'B_56.MOV', 'B_57.MOV', 'B_58.MOV', 'B_59.MOV', 'B_6.MOV', 'B_60.MOV', 'B_61.MOV', 'B_62.MOV', 'B_63.MOV', 'B_64.MOV', 'B_65.MOV', 'B_66.MOV

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



0: 544x960 2 items, 252.1ms
Speed: 7.0ms preprocess, 252.1ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 2 items, 178.1ms
Speed: 7.0ms preprocess, 178.1ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 (no detections), 177.1ms
Speed: 9.0ms preprocess, 177.1ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 (no detections), 164.1ms
Speed: 7.0ms preprocess, 164.1ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 (no detections), 175.1ms
Speed: 7.0ms preprocess, 175.1ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 item, 178.1ms
Speed: 8.0ms preprocess, 178.1ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 item, 159.1ms
Speed: 8.0ms preprocess, 159.1ms inference, 2.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 item, 140.6ms
Speed: 7.0ms preprocess, 140.6ms inference, 1.0ms postprocess 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



0: 544x960 1 item, 174.1ms
Speed: 7.0ms preprocess, 174.1ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 item, 180.5ms
Speed: 6.5ms preprocess, 180.5ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 (no detections), 183.6ms
Speed: 8.0ms preprocess, 183.6ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 item, 174.6ms
Speed: 8.0ms preprocess, 174.6ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 1 item, 169.6ms
Speed: 7.0ms preprocess, 169.6ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 (no detections), 168.6ms
Speed: 7.0ms preprocess, 168.6ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 (no detections), 169.1ms
Speed: 8.0ms preprocess, 169.1ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 960)

0: 544x960 (no detections), 173.1ms
Speed: 7.0ms preprocess, 173.1ms inference, 1.0ms postp